### import packages

In [1]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt

%matplotlib inline

### creating data directory 

In [2]:
data_dir = 'CE'

if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

In [3]:
print([f for f in os.listdir('.') if os.path.isfile(f)])

['CE_file_downloads.ipynb', '1CheezEyez draft.ipynb', 'metadata.ipynb', 'README.md', '.gitignore']


### don't need to run every time? qza file download

In [4]:
! wget -nv -O $data_dir/cheese-single-end-demux.qza 'https://polybox.ethz.ch/index.php/s/2AYhc9EgbtA7wsE/download'

2022-10-12 08:59:24 URL:https://polybox.ethz.ch/index.php/s/2AYhc9EgbtA7wsE/download [959420090/959420090] -> "CE/cheese-single-end-demux.qza" [1]


In [5]:
! ls -1 $data_dir

cheese-single-end-demux.qza


### tsv file download

In [6]:
! wget -nv -O $data_dir/food-metadata.tsv 'https://polybox.ethz.ch/index.php/s/nEd4l5CWGWGEtae/download'

2022-10-12 08:59:26 URL:https://polybox.ethz.ch/index.php/s/nEd4l5CWGWGEtae/download [42810/42810] -> "CE/food-metadata.tsv" [1]


In [7]:
! ls -1 $data_dir

cheese-single-end-demux.qza
food-metadata.tsv
